# Notebook for GPT-2 scratchwork.

In [3]:
import pandas as pd

dev_csv_path = '/Users/nicolewong/Desktop/nlp_urop/dev_lm_sentence_input.csv'

all_sentences = pd.read_csv(dev_csv_path)['user_candidate_transcription']
print(len(all_sentences))
num_select = 2
sentences_subset = list(all_sentences.iloc[:num_select])


3193


In [ ]:
subset_scores = gpt2_scores.score_sentences(sentences_subset, model_type = '')
print(subset_scores) # The kernel will die... too many sentences stored?

In [10]:
# 3/11: importlib help: https://stackoverflow.com/questions/1254370/reimport-a-module-in-python-while-interactive

import importlib
importlib.reload(bert_prefix_scores)

import bert_prefix_scores

bert_scores = bert_prefix_scores.score_sentences(sentences_subset)
print(bert_scores)

# You should now do the prefix predictions upwards -- what is faster? batching or?

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


[tensor([[[ -6.8940,  -6.8784,  -6.8544,  ...,  -6.0325,  -5.9080,  -3.9900],
         [-10.5577, -10.2678, -10.5460,  ..., -10.4262,  -8.6110,  -7.2549],
         [ -7.4243,  -7.6660,  -7.3358,  ...,  -4.8181,  -6.6352,  -7.0010],
         ...,
         [-14.7172, -14.3147, -14.3907,  ..., -13.4245, -11.5322,  -7.5067],
         [ -9.2270,  -9.0111,  -9.1821,  ...,  -7.5794,  -7.7827,  -6.0496],
         [-10.3993, -10.1649, -10.1911,  ...,  -8.8897,  -9.7768,  -6.9328]]]), tensor([[[ -7.0342,  -7.0048,  -6.9915,  ...,  -6.2103,  -6.0329,  -4.1197],
         [-10.2738, -10.0087, -10.2623,  ..., -10.0062,  -8.4222,  -7.2596],
         [-13.6738, -13.7455, -13.6967,  ..., -10.7754,  -9.9247, -11.1321],
         ...,
         [-16.3937, -16.0003, -16.0843,  ..., -15.0289, -12.7451,  -9.5425],
         [ -9.3667,  -9.0492,  -9.1858,  ...,  -8.0326,  -7.9546,  -6.4623],
         [-10.4887, -10.2320, -10.3047,  ...,  -9.2539, -10.1071,  -6.9714]]])]


In [23]:
# Check to see if the tokens exist in the current dataset

for sentence in all_sentences:
    for word in sentence.split(' '):
        if "##" == word[:2]:
            print(word)
            
print('done') # There are actually no tokens in the current sentence set.

done


In [ ]:
!python gpt2_tests.py